# Model Building With SMOTE

## Importing the libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.linear_model import LinearRegression
from numpy import mean
from numpy import absolute
from numpy import sqrt
import pandas as pd

## Importing the dataset

In [2]:
poll_df = pd.read_csv('air_pollution_gauss_cleaned.csv') # Loading normally or gaussian distributed data
poll_df.head()

,Date,State Name,New Covid Cases,Population Staying at Home,Population Not Staying at Home,Number of Trips,pm10_mean,pm25_mean,co_mean,no2_mean,o3_mean,so2_mean,press_mean,rh_mean,temp_mean,wind_mean,AQI,Category
0,2020-01-01,Alabama,0.0,859.830216,1710.371305,3245.716331,3.535534,2.868094,0.543782,4.046470,0.033676,0.951766,3.799426e+36,223.283219,48.435555,92.363889,6.055301,Good
1,2020-01-01,California,0.0,1161.114551,2234.236335,4629.817437,4.197069,3.454464,0.639386,3.185928,0.022262,0.755213,2.689899e+36,318.449161,49.710225,93.809963,7.440329,Moderate
2,2020-01-01,Colorado,0.0,766.009791,1408.948899,2977.793038,3.201562,2.187598,0.570414,3.518695,0.038290,1.050116,5.285798e+35,327.386469,31.375000,113.998580,5.524357,Good
3,2020-01-01,Connecticut,0.0,499.490741,975.321998,1965.252528,2.280351,1.762456,0.439804,2.559934,0.024059,0.325265,3.405333e+36,195.157135,37.482143,135.210606,4.708149,Good
4,2020-01-01,Florida,0.0,1160.936691,2275.216913,4589.595448,4.427189,3.655956,0.556034,3.329852,0.030658,1.010231,4.687015e+36,343.042315,61.408333,102.174306,6.786109,Good


## Applying SMOTE

In [3]:
from imblearn.over_sampling import SMOTE

X = poll_df[['New Covid Cases', 'Population Staying at Home',
       'Population Not Staying at Home', 'Number of Trips', 'pm10_mean',
       'pm25_mean', 'co_mean', 'no2_mean', 'o3_mean', 'so2_mean', 'press_mean',
       'rh_mean', 'temp_mean', 'wind_mean','AQI']].values
y = poll_df['Category'].values

smote =SMOTE(random_state=42)
X_sm,y_sm=smote.fit_resample(X,y)

In [4]:
poll_df['Category'].value_counts()

Good        20157
Moderate     2569
Name: Category, dtype: int64

In [5]:
pd.Series(y_sm).value_counts()

Good        20157
Moderate    20157
dtype: int64

## Selecting Dependent and Independent Features

In [6]:
X = X_sm[:,:-1]
y = X_sm[:,-1]

## Normalization and Scaling of Data

In [7]:
sc_X = StandardScaler()
sc_y = StandardScaler()

In [8]:
X = sc_X.fit_transform(X)

## Splitting the dataset into the Training set and Test set

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

## Regression Models On Balanced Dataset

### SVR model

#### Training

In [10]:
from sklearn.svm import SVR
regressor = SVR(kernel = 'rbf')
regressor.fit(X_train, y_train)

SVR()

#### Predicting the Test set results

In [11]:
from sklearn.preprocessing import StandardScaler
y_pred = regressor.predict(X_test)
np.set_printoptions(precision=2)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

[[7.31 7.2 ]
 [6.07 5.85]
 [7.61 7.48]
 ...
 [6.29 6.41]
 [7.45 7.45]
 [6.65 6.59]]


#### Model Evaluation

In [18]:
from sklearn.metrics import r2_score
print('MAE:', metrics.mean_absolute_error(y_test,y_pred))
print('MSE:', metrics.mean_squared_error(y_test, y_pred))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
print("R squared value: ",r2_score(y_test, y_pred))

MAE: 0.1830471526541624
MSE: 0.06514924714734811
RMSE: 0.2552435055928909
R squared value:  0.9345852091302579


### SVM regression using k fold cross validation and Grid Search

#### Training

In [13]:
from sklearn.svm import SVR
regressor = SVR(kernel = 'rbf')

#### 3 fold cross validation

In [14]:
cv=cross_validate(regressor, X, y, cv=3)

In [17]:
print('3 folds accuracy:',cv['test_score']*100)
print('mean accuracy of 3 folds:',cv['test_score'].mean()*100,'%')

3 folds accuracy: [78.57 91.12 18.  ]
mean accuracy of 3 folds: 62.562588630910454 %


#### SVM Grid Search

In [16]:
# defining parameter range
from sklearn.model_selection import GridSearchCV
param_grid = {'C': [0.1,1, 10, 100], 'gamma': [1,0.1,0.01,0.001],'kernel': ['rbf', 'poly', 'sigmoid']}
grid = GridSearchCV(regressor,param_grid,refit=True,verbose=2,cv = 3, n_jobs = -1)
grid.fit(X_train,y_train)

Fitting 3 folds for each of 48 candidates, totalling 144 fits
[CV] END .................C=0.1, gamma=0.001, kernel=sigmoid; total time= 1.8min
[CV] END ......................C=10, gamma=1, kernel=sigmoid; total time= 3.0min
[CV] END ......................C=0.1, gamma=0.1, kernel=poly; total time= 1.5min
[CV] END ......................C=1, gamma=0.001, kernel=poly; total time= 1.4min
[CV] END ..................C=10, gamma=0.001, kernel=sigmoid; total time= 1.9min
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time= 1.8min
[CV] END ......................C=10, gamma=1, kernel=sigmoid; total time= 3.0min
[CV] END ........................C=1, gamma=0.01, kernel=rbf; total time= 1.9min
[CV] END ....................C=10, gamma=0.1, kernel=sigmoid; total time= 3.1min
[CV] END .....................C=0.1, gamma=0.01, kernel=poly; total time= 1.9min
[CV] END ......................C=10, gamma=1, kernel=sigmoid; total time= 3.3min
[CV] END ........................C=1, gamma=0.0

[CV] END ....................C=0.1, gamma=0.001, kernel=poly; total time= 1.9min
[CV] END ........................C=10, gamma=0.1, kernel=rbf; total time=18.4min
[CV] END ......................C=0.1, gamma=0.01, kernel=rbf; total time= 1.9min
[CV] END .......................C=10, gamma=0.1, kernel=poly; total time=19.8min
[CV] END ......................C=0.1, gamma=0.01, kernel=rbf; total time= 1.9min
[CV] END .......................C=10, gamma=0.1, kernel=poly; total time=19.9min
[CV] END ..................C=0.1, gamma=0.01, kernel=sigmoid; total time= 1.9min
[CV] END .......................C=10, gamma=0.1, kernel=poly; total time=20.0min
[CV] END .....................C=1, gamma=0.1, kernel=sigmoid; total time= 3.4min
[CV] END .......................C=100, gamma=0.1, kernel=rbf; total time=60.2min
[CV] END .....................C=0.1, gamma=1, kernel=sigmoid; total time= 3.4min
[CV] END .......................C=100, gamma=0.1, kernel=rbf; total time=62.7min
[CV] END ...................

KeyboardInterrupt: 

#### Predicting Results

In [ ]:
from sklearn.preprocessing import StandardScaler
y_pred = grid.predict(X_test)
np.set_printoptions(precision=2)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

#### Evaluating Model

In [ ]:
from sklearn.metrics import r2_score
print('MAE:', metrics.mean_absolute_error(y_test,y_pred))
print('MSE:', metrics.mean_squared_error(y_test, y_pred))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
print("R squared value: ",r2_score(y_test, y_pred))

#### Plotting the results

In [ ]:
import seaborn as sns
plt.figure(figsize=(7,5))
sns.regplot(x=y_test,y=y_pred,ci=None,color ='red');
plt.title('Support Vector Regression with Grid Search')
plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')